In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
# import gc

stock_data = pd.read_csv('TOP_100_Stock_Name.csv', encoding='latin-1' , header=None)

type_of_data = 'dividend-yield-history'
urls = []
for i in range(1, len(stock_data)):
    urls.append('https://www.macrotrends.net/stocks/charts/' + str(stock_data[1][i]) + '/' + str(stock_data[0][i]) + '/' + str(type_of_data))

def parse_page(driver):
    driver.execute_script("window.scrollTo(0 , 1000)")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # g tag with this XPath : //*[@id="chartdiv"]/div/div/div[2]/div[3]/div/div/svg/g[12]/g
    # find div tag with this class name : amChartsPanel amcharts-stock-panel-div amcharts-stock-panel-div-stockPanel2
    main_div = soup.find_all('div', class_='amChartsPanel amcharts-stock-panel-div amcharts-stock-panel-div-stockPanel2')
    # find inner div tag with this class name : amcharts-chart-div
    inner_div = main_div[0].find_all('div', class_='amcharts-chart-div')
    # find g tag with this XPath : //*[@id="chartdiv"]/div/div/div[2]/div[3]/div/div/svg/g[12]/g
    g_tag = inner_div[0].find_all('g')[12]
    # find all circle tags
    circles = g_tag.find_all('circle')
    # Extract and print the aria-label attributes
    aria_labels = [circle['aria-label'] for circle in circles if 'aria-label' in circle.attrs]
    print(aria_labels)

    # save the data to a list
    data = []
    for label in aria_labels:
        data.append(label.split(' ')[0])
    return data
    


for count, url in enumerate(urls):
    filename = f'{type_of_data}\{stock_data[1][count + 1]}_{type_of_data}_Year_Dividend_history.csv'
    # Check if the file exists
    if not os.path.exists(filename):
        try: 
            time.sleep(2)
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--ignore-certificate-errors')
            chrome_options.add_argument('--disable-web-security')
            chrome_options.add_argument('--allow-running-insecure-content')
            driver = webdriver.Chrome()
            driver.implicitly_wait(4)
            driver.get(url)
            driver.maximize_window()
            # Wait for the page to load
            element = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.ID, "main_content"))
            )
            
            
            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0.jkvvVr"))).click()
                print('Click Accept All Button')
            except:
                print("No cookie consent pop-up found or failed to click 'Accept all'.")
        
            # Get the page source
            data = parse_page(driver)
            # Create a DataFrame
            df = pd.DataFrame(data)
            # Save the DataFrame to a CSV file
            df.to_csv(f'{type_of_data}\{stock_data[1][count + 1]}_{type_of_data}_Year_Dividend_history.csv', index=False , header=None)
            print("Save " + str(stock_data[1][count + 1]) + f" On {type_of_data}")
            # Close the browser
            driver.quit()
        except:
            print("Error On " + str(stock_data[1][count + 1]))
            driver.quit()
    else:
        print(stock_data[0][count + 1] + f"_{type_of_data}" + " already exist.")

In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to accept cookies
def accept_cookies(driver):
    try:
        accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0.jkvvVr")))
        if accept_button:
            accept_button.click()
            print("Clicked Accept All Button")
        else:
            print("Accept button not found")
    except Exception as e:
        print(f"Error clicking accept button: {e}")

# Function to scrape dividend data
def scrape_dividend_data(url):
    try:
        # Get the page content
        response = requests.get(url)
        response.raise_for_status()
        
        # Parse the page content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the relevant data using more specific selectors
        main_div = soup.find('div', {'id': 'main_content'})
        if not main_div:
            print("Main div not found")
            return None
        
        dividend_data = {}
        
        # Extract the current dividend payout
        payout_tag = main_div.find('meta', {'name': 'description'})
        if payout_tag:
            content = payout_tag.get('content')
            if content:
                # Assuming the format contains "dividend payout for Linde (LIN) as of <date> is <strong>$5.56</strong>"
                start_str = "dividend payout for Linde (LIN) as of"
                start_index = content.find(start_str)
                if start_index != -1:
                    data_str = content[start_index + len(start_str):].strip()
                    dividend_data['dividend_payout'] = data_str.split(' ')[-1].strip()
        
        # Extract the current dividend yield
        yield_tag = main_div.find('meta', {'name': 'description'})
        if yield_tag:
            content = yield_tag.get('content')
            if content:
                # Assuming the format contains "dividend yield for Linde as of <date> is <strong>1.28%</strong>"
                start_str = "dividend yield for Linde as of"
                start_index = content.find(start_str)
                if start_index != -1:
                    data_str = content[start_index + len(start_str):].strip()
                    dividend_data['dividend_yield'] = data_str.split(' ')[-1].strip()
        
        if not dividend_data:
            print("No data found for LIN")
        return dividend_data
    
    except requests.RequestException as e:
        print(f"Request error: {e}")
        return None
    except Exception as e:
        print(f"Error parsing data: {e}")
        return None

# Test URL
url = 'https://www.macrotrends.net/stocks/charts/LIN/linde/dividend-yield-history'
data = scrape_dividend_data(url)
if data:
    print("Dividend Data:", data)
else:
    print("Failed to retrieve dividend data")


Request error: 403 Client Error: Forbidden for url: https://www.macrotrends.net/stocks/charts/LIN/linde/dividend-yield-history
Failed to retrieve dividend data
